In [1]:
! venv\Scripts\activate.bat
! pip install -r requirements.txt

# Import PyRadios & Create instance of RadioBrowser

In [15]:
from pyradios import RadioBrowser
from geopy.geocoders import Nominatim
from random import choice
from IPython.display import FileLink

rb = RadioBrowser() # wrapper for the API at https://www.radio-browser.info/
geolocator = Nominatim(user_agent="TunerTerrace/1.0") # wrapper for https://nominatim.openstreetmap.org/ui/search.html

# Find Country/State/City associated with longitude and latitude

We'll try to find a station in Kathmandu:
27.7172° N, 85.3240° E

Note that geopy accepts all kinds of longitude/latitude formats. N/S  vs. positive/negative, include or don't include degrees are all fine.

In [27]:
def reverse_geo(long_lat, geolocator):

    location = geolocator.reverse(long_lat, language='en') 
    country = location.raw['address']['country']
    
    try:
        state = location.raw['address']['state']
    except KeyError:
        try:
            state = location.raw['address']['region']
        except KeyError:
            #sometimes cities are federal cities and have no state. Close enough
            state = location.raw['address']['city']
    if state == 'Autonomous City of Buenos Aires':
        state = 'Buenos Aires' #I don't have a a better way to deal with this haha

    try: 
        state_district = location.raw['address']['state_district']
    except KeyError:
        state_district = None
    try:
        city = location.raw['address']['city']
    except KeyError:
        city = None
    return [country, state, state_district, city]

In [30]:
print(reverse_geo('35°41′23″N 139°41′32″E', geolocator))
reverse_geo('27.7172° N, 85.3240° E', geolocator)

['Japan', 'Tokyo', None, 'Tokyo']


['Nepal', 'Bagmati Pradesh', None, 'Kathmandu']

# Find radio stations in/near that city

In [20]:
def get_stations(loc_ls, rb):
    country = loc_ls[0] 
    state = loc_ls[1] 
    state_district = loc_ls[2] 
    if loc_ls[3] == 'Autonomous City of Buenos Aires': # Super specific case I don't know how else to deal with
        city = 'Buenos Aires'
    else:
        city =loc_ls[3]
    # Find all stations in country
    sta_country = rb.stations_by_country(country=country)
    print("Number of Stations in Country: " + str(len(sta_country)))
    # If only one then you're done
    if len(sta_country) == 1:
        choice_ls = sta_country
    # If none then error
    elif len(sta_country)==0:
        raise('No Station Found')
    
    # Otherwise find all stations in state or state district
    else:
        sta_state = rb.stations_by_state(state=state) 
        if state_district:
            sta_state_dist = rb.stations_by_state(state=state_district) 
        else:
            sta_state_dist = []
        sta_state = sta_state + sta_state_dist
        
        # If more stations in the state then the country than something went wrong, return empty list
        if len(sta_state) > len(sta_country) or len(sta_state)==0:
            sta_state == []
            choice_ls = sta_country
        else:
            print("Number of Stations in State/Province/etc: " + (str(len(sta_state))))
            # If only one station in the state then you're done
            if len(sta_state) == 1:
                choice_ls = sta_state
            # Otherwise find all stations in the city
            # TODO: radio-browser data is not well-standardized and often city is listed in "country". Pull from there too
            # TODO: wholistically determine the search metrics such that most stations are found, with few false-positives
            else:
                sta_city = rb.stations_by_tag(tag=city) 
                if len(sta_city) > len(sta_state) or len(sta_city)==0:
                    sta_city = []
                    print("Number of Stations in City: " + (str(len(sta_city))))
                    choice_ls = sta_state
                else:
                    print("Number of Stations in City: " + (str(len(sta_city))))
                    choice_ls = sta_city
            
            
    return choice_ls

# Pick a station at random from the list

In [22]:
def pick_stations(choice_ls):
    play_me = choice(choice_ls)
    print(play_me['name'] + ' @:')

    print(play_me['url'])
    return play_me

# Put it all together...

In [25]:
loc_ls = reverse_geo('27.7172° N, 85.3240° E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

{'tourism': 'Narayanhiti Palace Museum', 'road': 'Hattisar Sadak', 'neighbourhood': 'Kamalpokhari', 'suburb': 'Narayan Chaur', 'city': 'Kathmandu', 'county': 'Kathmandu', 'region': 'Bagmati Pradesh', 'postcode': '44660', 'country': 'Nepal', 'country_code': 'np'}
Number of Stations in Country: 8
Kantipur FM @:
http://www.surfmusic.de/m3u/radio-kantipur-fm,13785.m3u


There are no stations tagged as being from Kathmandu specifically, but there are 3 stations in the State District that Kathmandu is in, Bagmati.  Therefore, we randomly select a station from that list of 3 stations.

# Try Adelaide, Australia

34.9285° S, 138.6007° E

In [48]:
loc_ls = reverse_geo('34.9285° S, 138.6007° E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 305
Number of Stations in State/Province/etc: 22
Number of Stations in City: 3
ABC Local Radio 891 Adelaide (AAC) @:
http://live-radio01.mediahubaustralia.com/5LRW/aac/


There are 4 stations tagged as being from Adelaide, so randomly pick one.

# Try Tonga!
21.1790° S, 175.1982° W

In [49]:
loc_ls = reverse_geo('21.1790° S, 175.1982° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 1
Radio Tonga 101.7 Nuku'alofa @:
http://tbcletio.serverroom.us:8142/;stream.mp3


This time there is 1 station in the whole country, so just return that one.

# Try Buenos Aires!

34.6037° S, 58.3816° W

In [24]:
loc_ls = reverse_geo('34.6037° S, 58.3816° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

{'house_number': '402', 'road': 'Cerrito', 'neighbourhood': 'Microcentro', 'suburb': 'San Nicolás', 'city': 'Autonomous City of Buenos Aires', 'postcode': 'C1043AAY', 'country': 'Argentina', 'country_code': 'ar'}
Number of Stations in Country: 321
Number of Stations in State/Province/etc: 105
Number of Stations in City: 4
Free FM Buenos Aires @:
http://pollux.shoutca.st:8968/


# Try LA!

In [51]:
loc_ls = reverse_geo('34.0522° N, 118.2437° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)


Number of Stations in Country: 3288
Number of Stations in State/Province/etc: 371
Number of Stations in City: 28
Echo Park Radio @:
http://104.63.241.231:8000/listen


This isn't totally perfect, because it returns a lot of stations for this densely populated area, and one of them is technically in compton, but it's still pretty good.

# Try Ghana!

7.9465° N, 1.0232° W

In [52]:
loc_ls = reverse_geo('7.9465° N, 1.0232° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 9
Asempa FM 94.7 Accra @:
http://stream.zenolive.com/esqh40m4wewtv


# Try Tunis! 
33.8869° N, 9.5375° E

In [61]:
loc_ls = reverse_geo('36°48′23″N 10°10′54″E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 31
Number of Stations in State/Province/etc: 6
Number of Stations in City: 0
Radio Jeunes @:
http://rtstream.tanitweb.com/jeunes


# Try Seattle!

In [54]:
loc_ls = reverse_geo('47.6062° N, 122.3321° W', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 3288
Number of Stations in State/Province/etc: 141
Number of Stations in City: 12
WestSeattle.Online/Radio @:
https://stream.radio.co/s80a8b3a06/listen


# Try Tokyo!

In [28]:
loc_ls = reverse_geo('35°41′23″N 139°41′32″E', geolocator)
choice_ls = get_stations(loc_ls, rb)
station = pick_stations(choice_ls)

Number of Stations in Country: 104
Number of Stations in State/Province/etc: 11
Number of Stations in City: 1
Free FM Tokyo @:
http://pollux.shoutca.st:8968/
